# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
df= pd.read_csv("event_datafile_new.csv")
df.query('itemInSession==4 & sessionId==338')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5250,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query= "CREATE TABLE IF NOT EXISTS song_details"
query= query+ "(itemInSession int, sessionId int, artist text, song text, length float, PRIMARY KEY(itemInSession,sessionId))"

try:
    session.execute(query)
except Exception as e:
    print(e)  

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
#         print(line[8],'\n',type(line[3]))
#         break
        query = "INSERT INTO song_details (itemInSession, sessionId, artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[3]),int(line[8]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query="SELECT * FROM song_details WHERE itemInSession=4 AND sessionId=338"
try:
    rows=session.execute(query)
except Exception as e:
    print(e) 
    
for row in rows:
    print(row.iteminsession, row.sessionid, row.artist, row.song, row.length)

4 338 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
df.query('userId==10 & sessionId==182')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6475,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
6476,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
6477,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
6478,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query= "CREATE TABLE IF NOT EXISTS song_by_user"
query= query+ "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId,sessionId),itemInSession))"
# query=query +"WITH CLUSTERING ORDER BY (sessionId ASC,itemInSession DESC)"
try:
    session.execute(query)
except Exception as e:
    print(e)  
                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
#         print(line[8],'\n',type(line[3]))
#         break
        query = "INSERT INTO song_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [15]:
query="SELECT * FROM song_by_user where userId=10 AND sessionid = 182"
try:
    rows=session.execute(query)
except Exception as e:
    print(e) 
    
for row in rows:
    print(row)

Row(userid=10, sessionid=182, iteminsession=0, artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(userid=10, sessionid=182, iteminsession=1, artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(userid=10, sessionid=182, iteminsession=2, artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(userid=10, sessionid=182, iteminsession=3, artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [16]:
df.query('song=="All Hands Against His Own"')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
885,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
2602,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
3864,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


In [17]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query= "CREATE TABLE IF NOT EXISTS all_user_by_song"
query= query+ "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song,userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)  

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
#         print(line[8],'\n',type(line[3]))
#         break
        query = "INSERT INTO all_user_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [20]:
query="SELECT song, firstName, lastName FROM all_user_by_song WHERE song='All Hands Against His Own'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e) 
    
for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
query="DROP TABLE song_by_user"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [ ]:
query="DROP TABLE all_user_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [ ]:
query="DROP TABLE song_details"
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()

In [ ]:
# test data
query="""
CREATE TABLE cycling.rank_by_year_and_name ( 
  race_year int, 
  race_name text, 
  cyclist_name text, 
  rank int, 
  PRIMARY KEY ((race_year, race_name), rank) );
"""
try:
    session.execute(query)
except Exception as e:
    print(e) 


In [ ]:
query="""

"""